In [5]:
from sklearnex import patch_sklearn
patch_sklearn()


import pandas as pd
import numpy as np
from sklearn.preprocessing import (
    Binarizer,
    StandardScaler,
    LabelBinarizer,
    OneHotEncoder,
)
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
# from xgboost import XGBRFClassifier, XGBClassifier
from sklearn.neural_network import MLPClassifier
# import dtale
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
# sns.set()
from sklearn import set_config
from sklearn.ensemble import StackingClassifier, HistGradientBoostingClassifier
from sklearn.gaussian_process.kernels import (
    Product,
    RBF,
    CompoundKernel,
    Exponentiation,
    Matern,
    Sum,
)
from sklearn.gaussian_process import GaussianProcessClassifier
from category_encoders import (
    TargetEncoder,
    BackwardDifferenceEncoder,
    BaseNEncoder,
    BinaryEncoder,
    CatBoostEncoder,
    CatBoostEncoder,
    GLMMEncoder,
    HelmertEncoder,
    JamesSteinEncoder,
    PolynomialEncoder,
    QuantileEncoder,
    SumEncoder,
    SummaryEncoder,
    WOEEncoder,
)
from sklearn.preprocessing import (
    StandardScaler,
    OneHotEncoder,
    OrdinalEncoder,
    MinMaxScaler,
)
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_graphviz
import warnings
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
import plotly.express as px
# from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import (
    RandomForestClassifier,
    ExtraTreesClassifier,
    AdaBoostClassifier,
    HistGradientBoostingClassifier,
    GradientBoostingClassifier,
)
from sklearn.experimental import enable_halving_search_cv
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import (
    train_test_split,
    cross_validate,
    # StratifiedGroupKFold,
    StratifiedKFold,
    GridSearchCV,
    RandomizedSearchCV,
    HalvingGridSearchCV,
    HalvingRandomSearchCV
)
import sigopt
from sklearn.naive_bayes import (
    BernoulliNB,
    CategoricalNB,
    MultinomialNB,
    ComplementNB,
    GaussianNB,
)
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import QuantileTransformer

# nb_est = [CategoricalNB(), MultinomialNB(), ComplementNB(), GaussianNB()]
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.tree import export_graphviz, plot_tree

# import seaborn as sns
# import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.feature_selection import (
    mutual_info_classif,
    SelectKBest,
    f_classif,
    chi2,
    RFE,
    SelectFdr,
    SelectFpr,
    SelectFwe,
    SelectPercentile,
)
from tbb import Monkey
# from xgboost import XGBClassifier
import os
from sklearn.svm import SVC, NuSVC
from sklearn.linear_model import PassiveAggressiveClassifier, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from joblib import parallel_backend
from category_encoders.wrapper import PolynomialWrapper, NestedCVWrapper
# from autosklearn.automl import AutoMLClassifier
import sklearn.metrics
pd.options.plotting.backend = "plotly"
pd.options.display.max_columns = 50
set_config(display="diagram")
warnings.filterwarnings("ignore")

dtype_info = {
    "v_1": "Binary",
    "v_26": "Binary",
    "v_11": "Binary",
    "v_14": "Binary",
    "v_30": "Binary",
    "v_28": "Binary",
    "v_9": "Binary",
    "v_27": "Binary",
    "v_32": "Nominal",
    "v_4": "Nominal",
    "v_3": "Nominal",
    "v_20": "Nominal",
    "v_21": "Nominal",
    "v_18": "Nominal",
    "v_25": "Nominal",
    "v_12": "Nominal",
    "v_31": "Ordinal",
    "v_15": "Ordinal",
    "v_19": "Ordinal",
    "v_13": "Ordinal",
    "v_33": "Ordinal",
    "v_17": "Ordinal",
    "v_29": "Ordinal",
    "v_23": "Ordinal",
    "v_6": "Ordinal",
    "v_24": "Ordinal",
    "v_10": "Ordinal",
    "v_5": "Ordinal",
    "v_22": "Ordinal",
    "v_0": "Ordinal",
    "v_16": "Ratio",
    "v_2": "Ratio",
    "v_8": "Ratio",
    "v_7": "Ratio",
    "v_39": "Ratio",
    "v_37": "Ratio",
    "v_38": "Ratio",
    "v_34": "Ratio",
    "v_40": "Ratio",
    "v_36": "Ratio",
    "v_35": "Ratio",
}
# data = pd.read_csv(
#     "../data/train.csv",
#     index_col=0,
# )
data__ = pd.read_parquet("../data/data_with_ridit.hdfs", engine="fastparquet")
prediction_data = pd.read_parquet("../data/test.parquet", engine="fastparquet")
data = pd.read_parquet("../data/train.parquet", engine="fastparquet")
ordinal = [i for i in dtype_info if dtype_info[i] == "Ordinal"]
nominal = [i for i in dtype_info if dtype_info[i] == "Nominal"]
binary = [i for i in dtype_info if dtype_info[i] == "Binary"]
ratio = [i for i in dtype_info if dtype_info[i] == "Ratio"]
final_data = pd.read_parquet("../data/final_data.parquet", engine="fastparquet")
final_pred_data = pd.read_parquet(
    "../data/final_pred_data.parquet", engine="fastparquet"
)
baseline_prediction_data = pd.read_parquet('../data/baseline.parquet')


def categorise_data(data):
    ordinal_data = data.loc[:, ordinal]
    nominal_data = data.loc[:, nominal]
    binary_data = data.loc[:, binary]
    ratio_data = data.loc[:, ratio]
    return ordinal_data, nominal_data, binary_data, ratio_data


ordinal_data, nominal_data, binary_data, ratio_data = categorise_data(final_data)
baseline_prediction_data.rename(columns={'label':'target'}, inplace=True)
final_data.rename(columns={'label':'target'}, inplace=True)
data.rename(columns={'label':'target'}, inplace=True)
# baseline_prediction_data

bs = pd.concat([final_data,baseline_prediction_data], axis=0,ignore_index=True)
nominal_master_db = bs.loc[:, nominal]

def gen_train_test(data, p):
    Y = data.target
    X_2 = Y_2 = Y[Y == 2].index
    X_0 = Y_0 = Y[Y == 0].index
    X_1 = Y_1 = Y[Y == 1].index
    train_size = int(p * Y_2.shape[0])
    test_size = int((1 - p) * Y_2.shape[0])

    train_idx_2 = np.random.choice(Y_2, (train_size,))
    train_idx_1 = np.random.choice(Y_1, (train_size,))
    train_idx_0 = np.random.choice(Y_0, (train_size,))
    train_idx = np.r_[train_idx_0, train_idx_1, train_idx_2]
    # train_idx.shape

    test_idx_2 = np.random.choice(np.setdiff1d(Y_2, train_idx_2), (test_size,))
    test_idx_1 = np.random.choice(np.setdiff1d(Y_1, train_idx_1), (test_size,))
    test_idx_0 = np.random.choice(np.setdiff1d(Y_0, train_idx_0), (test_size,))
    test_idx = np.r_[test_idx_0, test_idx_1, test_idx_2]
    # test_idx.shape
    return train_idx, test_idx

nominal_cont_map=defaultdict(dict)
nominal_indvl_map=defaultdict(dict)
for c in nominal:
    un = sorted(nominal_master_db[c].unique().tolist())
    n = len(un)
    new_id = list(range(n))
    nominal_indvl_map[c] = dict(zip(un,new_id))
start = 0
for c in nominal:
    un = sorted(nominal_master_db[c].unique().tolist())
    n = len(un)
    new_id = list(range(start,start+n))
    nominal_cont_map[c] = dict(zip(un,new_id))
    start += n


nominal_master_db_indvl = nominal_master_db.copy()
nominal_master_db_cont = nominal_master_db.copy()


# nominal_indvl_map
def nm_indvl_data_trnsform(row):
    for c in nominal:
        curr = row[c]
        row[c] = nominal_indvl_map[c][curr]
    return row


test1_nominal = nominal_master_db_indvl.apply(nm_indvl_data_trnsform,axis=1)


def nm_cont_data_trnsform(row):
    for c in nominal:
        curr = row[c]
        row[c] = nominal_cont_map[c][curr]
    return row


test2_nominal = nominal_master_db_cont.apply(nm_cont_data_trnsform,axis=1)


def best_n_features(n, X_train, y_train):
    ohe = OneHotEncoder(
        min_frequency=0.00001, handle_unknown="infrequent_if_exist", sparse=False
    )
    X_train_t = ohe.fit_transform(X_train)
    mic = mutual_info_classif(X_train_t, y_train, discrete_features=True)

    return ohe.get_feature_names_out()[mic.argsort()[-n:]]


# prediction_data = pd.read_pickle("../data/pred_data.pkl")
# est_ = [("cnb",CategoricalNB()),]
def iv_woe(data, target, bins=10, show_woe=False):

    # Empty Dataframe
    newDF, woeDF = pd.DataFrame(), pd.DataFrame()

    # Extract Column Names
    cols = data.columns

    # Run WOE and IV on all the independent variables
    for ivars in cols[~cols.isin([target])]:
        if (data[ivars].dtype.kind in "bifc") and (len(np.unique(data[ivars])) > 1000):
            binned_x = pd.qcut(data[ivars], bins, duplicates="drop")
            d0 = pd.DataFrame({"x": binned_x, "y": data[target]})
        else:
            d0 = pd.DataFrame({"x": data[ivars], "y": data[target]})
        d0 = d0.astype({"x": str})
        d = d0.groupby("x", as_index=False, dropna=False).agg({"y": ["count", "sum"]})
        d.columns = ["Cutoff", "N", "Events"]
        d["% of Events"] = np.maximum(d["Events"], 0.5) / d["Events"].sum()
        d["Non-Events"] = d["N"] - d["Events"]
        d["% of Non-Events"] = np.maximum(d["Non-Events"], 0.5) / d["Non-Events"].sum()
        d["WoE"] = np.log(d["% of Non-Events"] / d["% of Events"])
        d["IV"] = d["WoE"] * (d["% of Non-Events"] - d["% of Events"])
        d.insert(loc=0, column="Variable", value=ivars)
        print("Information value of " + ivars + " is " + str(round(d["IV"].sum(), 6)))
        temp = pd.DataFrame(
            {"Variable": [ivars], "IV": [d["IV"].sum()]}, columns=["Variable", "IV"]
        )
        newDF = pd.concat([newDF, temp], axis=0)
        woeDF = pd.concat([woeDF, d], axis=0)

        # Show WOE Table
        if show_woe == True:
            print(d)
    return newDF, woeDF

def wf_create(cat_encoder, model):
    encoder__name = cat_encoder.__class__.__name__
            # model = ComplementNB(norm=True,fit_prior=True,)
    # model = MultinomialNB()
    # model = LogisticRegression(n_jobs=-1, max_iter=10000,random_state=19)
    model__name = model.__class__.__name__
    pipe = Pipeline(steps=[('PW' + encoder__name, PolynomialWrapper(feature_encoder=cat_encoder)),
                           (model__name,model )])
    report = em.run(pipe) # Store A test log
    print('PW' + encoder_name, model__name)
    return pipe



# print(pipe)
# wf_create(TargetEncoder(),RandomForestClassifier())

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:
from pymongo import MongoClient
mdb = MongoClient()
# mdb.list_database_names()
class evaluate_model:
    collector = mdb.ml_results.cv_results
    
    
    def __init__(self, model, data=pd.DataFrame(), features=[]):
        self.model = model
        if data.shape[0]>0:
            self.data = data
        else:
            self.data = pd.read_parquet('../data/final_data.parquet', engine='fastparquet')
        if len(features)>0:
            self.features = features
        else:
            self.features = list(self.data.columns)
        self.train_idx, self.test_idx = gen_train_test(self.data, 1.0)
        self.X_train = self.data.loc[self.train_idx, self.features]
        self.y_train = self.data.target.loc[self.train_idx]
        self.cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=5)
        self.cv_results = None
        self.legacy=0
        
        
    def run(self, custom_model=None):
        if custom_model:
            selected_model = custom_model
        else:
            selected_model = self.model
        with parallel_backend('threading'):
            cv_results = cross_validate(selected_model,
                                        self.X_train,
                                        self.y_train,
                                        cv=self.cv,
                                        n_jobs=-1,
                                        scoring=['f1_micro',
                                                 'f1_macro',
                                                 'f1_weighted',
                                                 'precision_micro',
                                                 'precision_macro',
                                                 'precision_weighted',
                                                 'recall_micro',
                                                 'recall_macro',
                                                 'recall_weighted'],return_train_score=True,return_estimator=True)
                                        # error_score='raise',return_estimator=True)
                                    ## Save this results
        self.cv_results = cv_results
        # cv_db.insert_one(cv_results)
        # cnst = {'estimator_params': selected_model.get_params()}
        record =  {}
        record['estimator_params'] = str(selected_model.get_params())
        
        record['model_name'] = selected_model.__class__.__name__
        record['features'] = list(self.features)
        # record['data_idx'] = {'train_idx': list(self.train_idx),'test_idx': list(self.test_idx)}
        # cnst['fit_time'] = list(cv_results['fit_time'])
        for k,v in cv_results.items():
            if k in ['estimator','fit_time','score_time']:
                continue
            else:
                record[k] = v.mean()
        # for _ in cv_db.find():
        #     print(_.keys())
        # print(record)
        record['is_legacy_run'] = self.legacy
        record['legacy_report'] = None
        if self.legacy ==1:
            # print(self.legacy_run())
            l_rpt = self.legacy_run()
            for k, v in l_rpt.items():
                print(k)
                print('~'*20)
                print(v)
                print("#"*90)
            record['legacy_report'] = l_rpt
        
        collector.insert_one(record)
        
        return cv_results
    
    
    def plot(self):
        sns.set()
        fig, axes = plt.subplots(3, 3, figsize=(18, 15))        
        # fig.layout='constrained'

        metrics = ['f1_micro', 'f1_macro', 'f1_weighted', 'precision_micro', 'precision_macro', 'precision_weighted', 'recall_micro', 'recall_macro', 'recall_weighted']
        # for m  in metrics:
        #     n = len(self.cv_results['test_'+m])
        idx=0
        for i in range(3):
            n = len(self.cv_results['test_'+ metrics[0]])
            for j in range(3):
                axes[i,j].plot(range(n),self.cv_results['test_'+metrics[idx]],label=['test_'+metrics[idx]])
                # axes[i,j].plot(range(n),self.cv_results['train_'+metrics[idx]],label=['train_'+metrics[idx]])
                # axes[i,j].fill_between(range(n),self.cv_results['test_'+metrics[idx]],self.cv_results['train_'+metrics[idx]])
                axes[i,j].legend()
                axes[i,j].set_title(metrics[idx])
                # axes[i,j].autoscale(enable=False)
                idx+=1
        plt.legend()
    
    
    def plot_metric(self, metric):
        ydata = self.cv_results['test_'+metric]
        n = ydata.shape[0]
        ffig = plt.plot(np.arange(n),ydata,)
        fig = sns.regplot(x=np.arange(n), y=ydata)
        plt.title(metric.upper())
        return
    
    
    def switch_to_main_data(self):
        if self.legacy==1:
            print("#"*100)
            print("ALREADY ON LEGACY MODE")
            print("#"*100)
            return
        self.legacy = 1
        self.X_train_old = self.X_train
        self.y_train_old = self.y_train
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.data.loc[:,self.features], self.data.target)
        print("#"*100)
        print("SWITCHING TO LEGACY MODE")
        print("#"*100)
        
        
    def reset(self):
        if self.legacy==0:
            print("#"*100)
            print("NOT ON LEGACY MODE")
            print("#"*100)
            return
        self.legacy = 0
        self.X_train = self.X_train_old
        self.y_train = self.y_train_old
    
    
    def legacy_run(self):
        self.model.fit(self.X_train, self.y_train)
        self.y_pred = self.model.predict(self.X_test)
        self.legacy_report = {}
        self.legacy_report['classification_report'] = classification_report(self.y_test, self.y_pred)
        self.legacy_report['balanced_accuracy'] = sklearn.metrics.balanced_accuracy_score(self.y_test, self.y_pred)
        return self.legacy_report

    
    def print_last_run_stat(self):
        N = collector.estimated_document_count()
        idx = 0
        for c in collector.find(skip=N-1):
            for k,v in c.items():
                if 'f1_' in k and 'test' in k:
                    print(k)
                    print('~'*5)
                    print(v)
                    print('-'*20)
            print('#'*100)

In [37]:
labels = pd.get_dummies(bs.target)
target_classes = list(labels.columns)
# transformed_nominal = pd.DataFrame()
tr_nom_db = []
for t_c in target_classes:
    te = TargetEncoder(cols=nominal)
    tmp = te.fit_transform(test1_nominal,labels[t_c])
    tr_nom_db.append((transformed_nominal,labels[t_c]))
# RFECV()

# transformed_nominal.columns
# from helpers.wrappers import PolynomialWrapper as pw 
# pw_te = pw(te)
# del te_nominal_data
# pw_te.
# test1_nominal

# pw_te.fit(test1_nominal, bs.target)
# te_nom_mstr = pw_te.fit_transform(test1_nominal, pd.get_dummies(bs.target))
# te_nominal_data = pw_te.fit_transform(test1_nominal.loc[:,nominal],bs.target)
# pwt_data['target'] = final_data.target
# pwt_data
# te_nom_mstr.columns
# te_nominal_data

In [38]:
from sklearnex.cluster import DBSCAN, KMeans     
from sklearn.feature_selection import RFECV


In [ ]:
from category_encoders import MEstimateEncoder


ovr_pipe = Pipeline(steps=[(encoder_name,cat_encoder), (model__name, model)])
                           
# ovr_pipe
ovr = OneVsRestClassifier(ovr_pipe)
# ovr
# pipe
# print(pipe)
d_coll = []
try:
    for m_ in tqdm(np.linspace(0.1,1.5,num=20)):
        cat_encoder = TargetEncoder(drop_invariant=False,cols=['v_32','v_4','v_3','v_20','v_21','v_18','v_25','v_12'],smoothing=m_)
        encoder_name = cat_encoder.__class__.__name__
        # model = ComplementNB(norm=True,fit_prior=True,)
        model = MultinomialNB()
        model = LogisticRegression(n_jobs=-1, max_iter=10000,random_state=19)
        model__name = model.__class__.__name__
        pipe = Pipeline(steps=[('PW' + encoder_name, PolynomialWrapper(feature_encoder=cat_encoder)),
                               (model__name,model )])

        report = em.run(pipe)
        print(report['test_f1_macro'].mean())
        d_coll.append((m_,report['test_f1_macro'].mean(), report['test_f1_macro'].std(), report['test_f1_macro'].max()))
except KeyboardInterrupt:
    print("Interrupted, Plotting calculated values only")
# pipe = make
def plot_m_variance(d_coll):
    sns.set()
    ddx= [x for x,y,u,r in d_coll]
    ddc = [y for x,y,u,r in d_coll]
    ddep = [y+u for x,y,u,r in d_coll]
    dden = [y-u for x,y,u,r in d_coll]
    ddem = [r for x,y,u,r in d_coll]
    plt.plot(ddx,ddc,'b',label="\u00b5")
    plt.plot(ddx,ddep,'r',label="\u03c3"+"+")
    plt.plot(ddx,dden,'g',label="\u03c3"+"-")
    plt.plot(ddx,ddem,'y',label="\u03c3"+"max")
    fig=plt.fill_between(ddx,ddep,dden,alpha=0.5)
    fig = plt.legend()
plot_m_variance(d_coll)

In [15]:
bs.target

0       1
1       2
2       1
3       1
4       1
       ..
5419    0
5420    1
5421    0
5422    1
5423    1
Name: target, Length: 5424, dtype: int64